In [1]:
import sys
import numpy as np
import cv2
from matplotlib import pyplot as plt

In [2]:
# import os
# from google.colab import files
# uploaded = files.upload()
# dst ='image.jpg' 
# os.rename(list(uploaded.keys())[0], dst)

In [3]:
# import os
# from google.colab import files
# uploaded = files.upload()
# dst ='movie.mp4' 
# os.rename(list(uploaded.keys())[0], dst)

In [4]:
# damage colors(BGR)
elementColors = dict()
elementColors['anemo'] = (200, 255, 106)
elementColors['geo'] = (101, 204, 254)
elementColors['electro'] = (255, 155, 224)
elementColors['physical'] = (255, 255, 255)
elementColors['pyro'] = (3, 155, 254)
elementColors['cyro'] = (255, 252, 155)
elementColors['hydro'] = (253, 204, 52)

resizeScale = 0.5
colorKey = "cyro"

In [8]:
imageCounter = 0
def outputNumberBinaryImage(im, rect):
  global imageCounter
  x,y,w,h = rect
  outputImage = im[y:y+h, x:x+w]
  closing = cv2.dilate(outputImage,(4, 4),iterations = 1)
  cv2.imwrite("./output/{}_{}.png".format(colorKey, imageCounter), closing)
  imageCounter += 1

In [9]:
def rectangleScaled(im, start, end, color, thickness=1):
    start = (int(start[0] / resizeScale), int(start[1] / resizeScale))
    end = (int(end[0] / resizeScale), int(end[1] / resizeScale))
    cv2.rectangle(im, start, end, color, thickness)

In [10]:


def drawContours(im, resized, parcent, width, height):
  global prevFrameNumber, resizeScale, textpos
  
  rangeW = 25
  color = elementColors[colorKey]
  inrange = cv2.inRange(resized, (max(color[0] - rangeW, 0), max(color[1] - rangeW, 0), max(color[2] - rangeW, 0)), (min(255, color[0] + rangeW), min(255, color[1] + rangeW), min(255, color[2] + rangeW)))
  # for key, color in elementColors.items():
  #   nextInrange = cv2.inRange(im, (color[0] - rangeW, color[1] - rangeW, color[2] - rangeW), (color[0] + rangeW, color[1] + rangeW, color[2] + rangeW))
  #   inrange = np.maximum(nextInrange, inrange)

  # blur = cv2.GaussianBlur(inrange, (5, 5), 0)
  thresh = cv2.adaptiveThreshold(inrange, 255, 1, 1, 7, 2)

  contours, _ = cv2.findContours(
      thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  
  frameContours = []
  
  nextFrameNumber = []

  for cnt in contours:
    rect = cv2.boundingRect(cnt.astype(np.int32))
    x, y, w, h = rect
    frameContours.append((x + w / 2, y + h / 2))
    

    if h < 2 or w / (h * 0.8) < 3:
      continue
    
    outputNumberBinaryImage(inrange, rect)
    red = (0, 0, 255)
    rectangleScaled(im, (x, y), (x + w, y + h), red, 2)


  return im

In [12]:
filename = "tartagria"
cap = cv2.VideoCapture('./{}.mp4'.format(filename))
fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
frame_rate = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter('./{}_output.mp4'.format(filename), fourcc, frame_rate, (width, height))
colorKey = "hydro"
prevFrameNumber = []
import queue
textQueue = queue.Queue()

count = 0
imageCounter = 0

frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

while(cap.isOpened()):
  ret, frame = cap.read()
  count += 1
  if count % 24 == 0:
    print(count / frame_count * 100, '%')
  if (not(ret)):
    break
  
  color = elementColors[colorKey]
  rangeW = 30
  resized = cv2.resize(frame, (int(frame.shape[1] * resizeScale), int(frame.shape[0] * resizeScale)))
  res = drawContours(frame, resized, count / frame_count, frame.shape[1], frame.shape[0])

    
  # inrange = cv2.inRange(resized, (max(color[0] - rangeW, 0), max(color[1] - rangeW, 0), max(color[2] - rangeW, 0)), (min(255, color[0] + rangeW), min(255, color[1] + rangeW), min(255, color[2] + rangeW)))
  # # for key, color in elementColors.items():
  # #   nextInrange = cv2.inRange(im, (color[0] - rangeW, color[1] - rangeW, color[2] - rangeW), (color[0] + rangeW, color[1] + rangeW, color[2] + rangeW))
  # #   inrange = np.maximum(nextInrange, inrange)

  # # blur = cv2.GaussianBlur(inrange, (5, 5), 0)
  # res = cv2.adaptiveThreshold(inrange, 255, 1, 1, 7, 2)

  # gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
  out.write(res)
  cv2.imshow("frame", res)
  
  if cv2.waitKey(1) & 0xFF == ord('q'):
      break

out.release()
cap.release()
cv2.destroyAllWindows()
    


0.11920727164357026 %
0.2384145432871405 %
0.35762181493071077 %
0.476829086574281 %
0.5960363582178513 %
0.7152436298614215 %
0.8344509015049918 %
0.953658173148562 %
1.0728654447921324 %
1.1920727164357026 %
1.311279988079273 %
1.430487259722843 %
1.5496945313664132 %
1.6689018030099836 %
1.7881090746535537 %
1.907316346297124 %
2.0265236179406942 %
2.145730889584265 %
2.264938161227835 %
